<div style="background: linear-gradient(to right, #4b6cb7, #182848); padding: 20px; border-radius: 10px; text-align: center; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
    <h1 style="color: white; margin: 0; font-size: 2.5em; font-weight: 700;">GAICo: GenAI Results Comparator</h1>
    <p style="color:hsl(0, 0.00%, 87.80%); margin-top: 10px; font-style: italic; font-size: 1.2em; text-align: center;">A Hands-On on Evaluation w/ Thresholds</p>
</div>
<br>

Welcome to the <b>GAICo</b> library! This notebook will guide you through its features and demonstrate how to use it for evaluating text generated by Large Language Models (LLMs) against reference texts. We'll cover everything from basic setup to calculating various metrics and applying thresholds to the results.

This example is designed for technical users who want to:

- Compare generated text strings with ground truth versions.
- Utilize a range of reference-based evaluation metrics.
- Process single pairs or batches of text efficiently.
- Applying default & custom thresholds to the results.


### 1. Import Required Libraries


For this notebook, we'll assume the library is installed or made accessible via the path modification below.


In [1]:
import sys
import os

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Construct the path to the project root (two levels up)
project_root = os.path.abspath(os.path.join(notebook_dir, os.pardir, os.pardir))

# Add project root to the system path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

Now, that the `gaico` package is confirmed importable, let's continue by importing necessary modules from the library and other common packages.


In [2]:
import pandas as pd

from gaico.metrics import (
    BLEU,
    ROUGE,
    JSDivergence,
    JaccardSimilarity,
    CosineSimilarity,
    LevenshteinDistance,
    SequenceMatcherSimilarity,
    BERTScore,
)

# Import below can also be called from `gaico` directly
from gaico.utils import generate_deltas_frame
from gaico.thresholds import (
    apply_thresholds,
    get_default_thresholds,
    calculate_pass_fail_percent,
)

print("Libraries imported successfully!")

Libraries imported successfully!


### 2. Initialize Metrics

We will initialize all metric instances as a dictionary with names as keys.

In [ ]:
metrics = {
    "BLEU": BLEU(),
    "ROUGE": ROUGE(rouge_types=["rougeL"]),  # Using ROUGE-L for simplicity
    "JSD": JSDivergence(),
    "BERTScore": BERTScore(model_type="bert-base-uncased", output_val=["f1"]),
    "Jaccard": JaccardSimilarity(),
    "Cosine": CosineSimilarity(),
    "Levenshtein": LevenshteinDistance(),
    "SequenceMatcher": SequenceMatcherSimilarity(),
}

print("Metrics initialized:")
for name in metrics.keys():
    print(f"  - {name}")

Metrics initialized:
  - BLEU
  - ROUGE
  - JSD
  - BERTScore
  - Jaccard
  - Cosine
  - Levenshtein
  - SequenceMatcher


### 3. Sample Data for the implementation


In [4]:
# Single pair example
single_generated = "The quick brown fox jumps over the lazy dog"
single_reference = "A quick brown fox jumped over a lazy dog"

# Batch examples - varying quality of translations/paraphrases
generated_texts = [
    # High quality paraphrases
    "The quick brown fox jumps over the lazy dog",
    "A beautiful sunset painted the sky with vibrant colors",
    "Machine learning models require large amounts of data",
    # Medium quality paraphrases
    "The cat sat on the comfortable mat",
    "Artificial intelligence is transforming many industries",
    # Lower quality / more different
    "Dogs are loyal companions",
    "Programming requires logical thinking",
    "The weather is nice today",
]

reference_texts = [
    # Corresponding references
    "A quick brown fox jumped over a lazy dog",
    "The sunset painted beautiful colors across the sky",
    "Machine learning algorithms need substantial data for training",
    "The cat was sitting on the mat",
    "AI technology is revolutionizing various sectors",
    "Cats make independent pets",
    "Coding needs systematic problem-solving skills",
    "It's a pleasant day outside",
]

### 4. Test Case-1: Single Pair Evaluation


In [5]:
print("=== SINGLE PAIR EVALUATION ===\n")

# Calculate scores for single pair
single_scores = {}
for name, metric in metrics.items():
    score = metric.calculate(single_generated, single_reference)
    single_scores[name] = score
    print(f"{name}: {score:.4f}" if isinstance(score, (int, float)) else f"{name}: {score}")

# Apply thresholds
print("\n** Applying Default Thresholds **")
default_thresholds = get_default_thresholds()
print("Default thresholds:", default_thresholds)

single_results = apply_thresholds(single_scores)

# Display results
print("\n** Threshold Results **")
for metric_name, details in single_results.items():
    print(f"\n{metric_name}:")
    print(
        f"  Score: {details['score']:.4f}"
        if isinstance(details["score"], (int, float))
        else f"  Score: {details['score']}"
    )
    print(f"  Threshold: {details['threshold_applied']}")
    print(f"  Passed: {details['passed_threshold']}")

=== SINGLE PAIR EVALUATION ===

BLEU: 0.1562
ROUGE: 0.7778
JSD: 0.5193
BERTScore: 0.9103
Jaccard: 0.6000
Cosine: 0.6838
Levenshtein: 0.8675
SequenceMatcher: 0.8675

** Applying Default Thresholds **
Default thresholds: {'BLEU': 0.5, 'ROUGE': 0.5, 'JSD': 0.5, 'BERTScore': 0.5, 'Jaccard': 0.5, 'Cosine': 0.5, 'Levenshtein': 0.5, 'SequenceMatcher': 0.5}

** Threshold Results **

BLEU:
  Score: 0.1562
  Threshold: 0.5
  Passed: False

ROUGE:
  Score: 0.7778
  Threshold: 0.5
  Passed: True

JSD:
  Score: 0.5193
  Threshold: 0.5
  Passed: False

BERTScore:
  Score: 0.9103
  Threshold: 0.5
  Passed: True

Jaccard:
  Score: 0.6000
  Threshold: 0.5
  Passed: True

Cosine:
  Score: 0.6838
  Threshold: 0.5
  Passed: True

Levenshtein:
  Score: 0.8675
  Threshold: 0.5
  Passed: True

SequenceMatcher:
  Score: 0.8675
  Threshold: 0.5
  Passed: True


### 5. Test Case-2: Batch Evaluation


In [6]:
print("\n=== BATCH EVALUATION ===\n")

# Calculate scores for batch
batch_scores = []
for i in range(len(generated_texts)):
    item_scores = {}
    for name, metric in metrics.items():
        item_scores[name] = metric.calculate(generated_texts[i], reference_texts[i])
    batch_scores.append(item_scores)

print(f"Calculated scores for {len(batch_scores)} text pairs")

# Apply thresholds to batch
batch_results = apply_thresholds(batch_scores)

# Display first few results
print("\n** Sample Results (First 3 items) **")
for i in range(min(3, len(batch_results))):
    print(f"\nItem {i}:")
    print(f"Generated: '{generated_texts[i][:50]}...'")
    print(f"Reference: '{reference_texts[i][:50]}...'")
    for metric_name, details in batch_results[i].items():
        score_str = (
            f"{details['score']:.3f}"
            if isinstance(details["score"], (int, float))
            else str(details["score"])
        )
        passed = "✓" if details["passed_threshold"] else "✗"
        print(f"  {metric_name}: {score_str} {passed}")


=== BATCH EVALUATION ===

Calculated scores for 8 text pairs

** Sample Results (First 3 items) **

Item 0:
Generated: 'The quick brown fox jumps over the lazy dog...'
Reference: 'A quick brown fox jumped over a lazy dog...'
  BLEU: 0.156 ✗
  ROUGE: 0.778 ✓
  JSD: 0.519 ✗
  BERTScore: 0.910 ✓
  Jaccard: 0.600 ✓
  Cosine: 0.684 ✓
  Levenshtein: 0.867 ✓
  SequenceMatcher: 0.867 ✓

Item 1:
Generated: 'A beautiful sunset painted the sky with vibrant co...'
Reference: 'The sunset painted beautiful colors across the sky...'
  BLEU: 0.079 ✗
  ROUGE: 0.471 ✗
  JSD: 0.549 ✗
  BERTScore: 0.790 ✓
  Jaccard: 0.600 ✓
  Cosine: 0.783 ✓
  Levenshtein: 0.519 ✓
  SequenceMatcher: 0.462 ✗

Item 2:
Generated: 'Machine learning models require large amounts of d...'
Reference: 'Machine learning algorithms need substantial data ...'
  BLEU: 0.065 ✗
  ROUGE: 0.375 ✗
  JSD: 0.342 ✓
  BERTScore: 0.765 ✓
  Jaccard: 0.231 ✗
  Cosine: 0.375 ✗
  Levenshtein: 0.557 ✓
  SequenceMatcher: 0.522 ✓


### 6. Calculating Pass/Fail Percentages


In [7]:
print("\n=== PASS/FAIL STATISTICS ===\n")

# Extract scores in the format expected by calculate_pass_fail_percentages
scores_dict = {}
for metric_name in metrics.keys():
    scores_dict[metric_name] = [item[metric_name] for item in batch_scores]

# Calculate percentages
percentages = calculate_pass_fail_percent(scores_dict)

# Display results
print("Metric Performance Summary:")
print("-" * 60)
print(f"{'Metric':<15} {'Pass Rate':<12} {'Passed':<10} {'Failed':<10}")
print("-" * 60)

for metric, stats in percentages.items():
    print(
        f"{metric:<15} {stats['pass_percentage']:>6.1f}% {stats['total_passed']:>10} {stats['total_failed']:>10}"
    )


=== PASS/FAIL STATISTICS ===

Metric Performance Summary:
------------------------------------------------------------
Metric          Pass Rate    Passed     Failed    
------------------------------------------------------------
BLEU               0.0%          0          8
ROUGE             25.0%          2          6
JSD               62.5%          5          3
BERTScore        100.0%          8          0
Jaccard           37.5%          3          5
Cosine            37.5%          3          5
Levenshtein       50.0%          4          4
SequenceMatcher   37.5%          3          5


### 7. Generating CSV report for the deltas


In [8]:
# Single pair CSV
generate_deltas_frame(
    single_results,
    generated_texts=single_generated,
    reference_texts=single_reference,
    output_csv_path=os.path.join("..", "data", "threshold", "single_pair_analysis.csv"),
)

# Batch CSV
generate_deltas_frame(
    batch_results,
    generated_texts=generated_texts,
    reference_texts=reference_texts,
    output_csv_path=os.path.join("..", "data", "threshold", "batch_analysis.csv"),
)

# Display a sample of the batch CSV
print("\nSample of batch analysis CSV:")
df = pd.read_csv(os.path.join("..", "data", "threshold", "batch_analysis.csv"))
print(df.head(3))

CSV report generated at: ../data/threshold/single_pair_analysis.csv
CSV report generated at: ../data/threshold/batch_analysis.csv

Sample of batch analysis CSV:
                                      generated_text  \
0        The quick brown fox jumps over the lazy dog   
1  A beautiful sunset painted the sky with vibran...   
2  Machine learning models require large amounts ...   

                                      reference_text  BLEU_score  BLEU_passed  \
0           A quick brown fox jumped over a lazy dog    0.156197        False   
1  The sunset painted beautiful colors across the...    0.079369        False   
2  Machine learning algorithms need substantial d...    0.065006        False   

   ROUGE_score  ROUGE_passed  JSD_score  JSD_passed  BERTScore_score  \
0     0.777778          True   0.519324       False         0.910302   
1     0.470588         False   0.549008       False         0.790194   
2     0.375000         False   0.341808        True         0.765008   



### 8. Testing with Custom Thresholds


In [9]:
# Defining custom thresholds
custom_thresholds = {
    "BLEU": 0.7,
    "ROUGE": 0.65,
    "JSD": 0.3,  # Lower is better for JSD
    "BERTScore": 0.85,
    "Jaccard": 0.7,
    "Cosine": 0.8,
    "Levenshtein": 0.7,
    "SequenceMatcher": 0.75,
}

print("Custom (stricter) thresholds:")
for metric, threshold in custom_thresholds.items():
    print(f"  {metric}: {threshold}")

# Apply custom thresholds
custom_results = apply_thresholds(batch_scores, thresholds=custom_thresholds)

# Calculate new percentages
custom_percentages = calculate_pass_fail_percent(scores_dict, thresholds=custom_thresholds)

print("\n** Comparison: Default vs Custom Thresholds **")
print(f"{'Metric':<15} {'Default Pass %':<15} {'Custom Pass %':<15} {'Difference':<10}")
print("-" * 55)

for metric in metrics.keys():
    default_pass = percentages[metric]["pass_percentage"]
    custom_pass = custom_percentages[metric]["pass_percentage"]
    diff = custom_pass - default_pass
    print(f"{metric:<15} {default_pass:>13.1f}% {custom_pass:>13.1f}% {diff:>+9.1f}%")

# Generate CSV with custom thresholds
deltas_frame = generate_deltas_frame(
    custom_results,
    generated_texts=generated_texts,
    reference_texts=reference_texts,
    output_csv_path=os.path.join("..", "data", "threshold", "custom_threshold_analysis.csv"),
)

print("\nTable showing the results:")
display(deltas_frame)

Custom (stricter) thresholds:
  BLEU: 0.7
  ROUGE: 0.65
  JSD: 0.3
  BERTScore: 0.85
  Jaccard: 0.7
  Cosine: 0.8
  Levenshtein: 0.7
  SequenceMatcher: 0.75

** Comparison: Default vs Custom Thresholds **
Metric          Default Pass %  Custom Pass %   Difference
-------------------------------------------------------
BLEU                      0.0%           0.0%      +0.0%
ROUGE                    25.0%          25.0%      +0.0%
JSD                      62.5%         100.0%     +37.5%
BERTScore               100.0%          25.0%     -75.0%
Jaccard                  37.5%           0.0%     -37.5%
Cosine                   37.5%           0.0%     -37.5%
Levenshtein              50.0%          12.5%     -37.5%
SequenceMatcher          37.5%          12.5%     -25.0%
CSV report generated at: ../data/threshold/custom_threshold_analysis.csv

Table showing the results:


,generated_text,reference_text,BLEU_score,BLEU_passed,ROUGE_score,ROUGE_passed,JSD_score,JSD_passed,BERTScore_score,BERTScore_passed,Jaccard_score,Jaccard_passed,Cosine_score,Cosine_passed,Levenshtein_score,Levenshtein_passed,SequenceMatcher_score,SequenceMatcher_passed
0,The quick brown fox jumps over the lazy dog,A quick brown fox jumped over a lazy dog,0.156197,False,0.777778,True,0.519324,True,0.910302,True,0.600000,False,0.683763,False,0.867470,True,0.867470,True
1,A beautiful sunset painted the sky with vibran...,The sunset painted beautiful colors across the...,0.079369,False,0.470588,False,0.549008,True,0.790194,False,0.600000,False,0.782624,False,0.519231,False,0.461538,False
2,Machine learning models require large amounts ...,Machine learning algorithms need substantial d...,0.065006,False,0.375000,False,0.341808,True,0.765008,False,0.230769,False,0.375000,False,0.556522,False,0.521739,False
3,The cat sat on the comfortable mat,The cat was sitting on the mat,0.104455,False,0.714286,True,0.554981,True,0.853967,True,0.500000,False,0.777778,False,0.656250,False,0.656250,False
4,Artificial intelligence is transforming many i...,AI technology is revolutionizing various sectors,0.040825,False,0.166667,False,0.239985,True,0.743125,False,0.090909,False,0.166667,False,0.446602,False,0.446602,False
5,Dogs are loyal companions,Cats make independent pets,0.000000,False,0.000000,False,0.167445,True,0.631503,False,0.000000,False,0.000000,False,0.352941,False,0.313725,False
6,Programming requires logical thinking,Coding needs systematic problem-solving skills,0.000000,False,0.000000,False,0.167445,True,0.548517,False,0.000000,False,0.000000,False,0.385542,False,0.361446,False
7,The weather is nice today,It's a pleasant day outside,0.000000,False,0.000000,False,0.167445,True,0.617866,False,0.000000,False,0.000000,False,0.346154,False,0.307692,False


### 9. Conclusion

This notebook has demonstrated the core functionalities of the `GAICo` library. You've learned how to:

- Initialize and use various N-gram, text similarity, and semantic similarity metrics.
- Calculate scores for single text pairs and batches of texts.
- Evaluate the results on various default and custom threshold settings.

The library is designed to be extensible, so you can also create your own custom metrics by inheriting from `BaseMetric`. For more advanced use cases, such as prompt-aware evaluation, check the `examples` folder in the library's repository.

We encourage you to explore the library further with your own datasets and LLM outputs. Happy evaluating!
